<h1>Suggested Process</h1>
<h2>Abstract</h2>
<p>Este cuaderno esta enfocado en organizar la información para construir el archivo de sugerido de compra para la geografía de Europa</p>
<h2>Trabajar con archivos de inventario</h2>
<p>En este punto vamos a cargar las librerías para trabajar con data:</p>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_columns = None
from datetime import datetime, timedelta
import calendar
import cx_Oracle
import json
import pulp as pu
from tqdm import tqdm
import concurrent.futures
import time
import os

In [2]:
# generar la fecha de hoy
hoy_date = datetime(datetime.now().year,datetime.now().month, datetime.now().day) 
hoy = datetime.now().strftime("%Y%m%d")

<h2>Parámetros generales</h2>
<p>Los siguientes son los clientes a quien haremos sugerido</p>

In [3]:
pqn_customers = ['101111', '100459', '101724']

<h2>Descargar pedidos de Europa (Beta)</h2>
<p>Descarga los pedidos de Europa desde la consulta en global.</p>

In [4]:
with open('D:\\OneDrive - Proquinal\\keys.json') as file:
    keys = json.load(file)
cad_con= f'{keys["global_user_noexpirer_user"]}/{keys["global_user_noexpirer_pass"]}@172.16.1.3/global'

In [5]:
# Pedidos
sql_query = 'Select * From Global.V_Glb_Pedidos_Eur'
with cx_Oracle.connect(cad_con) as con:
    pedidos_df = pd.read_sql(sql=sql_query, con=con)

In [ ]:
# Inventarios
sql_query = 'Select * From Global.V_GLB_INVENTARIO_EUR'
with cx_Oracle.connect(cad_con) as con:
    inventarios_global_df = pd.read_sql(sql=sql_query, con=con)

In [ ]:
inventarios_global_df['FECHA_CORTE'].unique()

In [ ]:
# Transitos Pendientes


In [ ]:
# Categorias
sql_query = 'Select * From Global.V_GLB_CATEGO_EUR'
with cx_Oracle.connect(cad_con) as con:
    productos_global_df = pd.read_sql(sql=sql_query, con=con)

In [ ]:
productos_global_df[productos_global_df['GRUPOART'].isin(['Service', 'Services', 'ACTIVOS FIJOS'])]

In [ ]:
pedidos_df.groupby(['EMPRESA'])[['OPENQTY']].sum()

In [ ]:
pedidos_df[].head()

In [ ]:
#pedidos_df['ITEMCODE'].unique()
#sorted(list(pedidos_df['DOCDUEDATE'].unique()))
#sorted(list(pedidos_df['DOCDATE'].unique()), reverse=True)
#pedidos_df['OPENQTY'].hist()
pedidos_df[pedidos_df['CARDNAME']=='MSC CRUISE MANEGEMENT (UK) lTD.'].to_excel('borrame.xlsx')

<h3>Preguntas para Reunión de seguimiento</h3>
<ol>
    <li>¿Se puede identificar el cliente? (Util para seguimiento del forecast) rta: se va a agregar</li>
    <li>¿Se puede tener el código Stock en esta consulta? rta: Listo</li>
    <li>¿Se puede obtener el número de las exportaciones que están en tránsito? ¿cómo los identifican Emma, Cristina y Sandra? rta: sí, se agregará</li>
    <li>si DOCDUEDATE es la fecha de compromiso, ¿Se pueden tener como urgencia las fechas anteriores a la actual?    rta: (Consultar con Cristina, Por ahora no tener en cuenta estos pedidos ALeco)</li>
</ol>

<h2>Leer archivos de ventas reportados por la geografía</h2>
<p>Vamos a empezar leyendo los archivos de inventarios que reporta cada geografía. Posteriormente, migraremos a emplear los archivos traídos desde global</p>

In [6]:
gmb_inventory= pd.read_excel('GMB.xlsx')
print(gmb_inventory.shape)
# remover vacios
gmb_inventory = gmb_inventory[~gmb_inventory['Stock Code (CU)'].isna()]
print(gmb_inventory.shape)
# Totalizar por Stockcode
gmb_inventory = gmb_inventory.groupby(['Stock Code (CU)'])[['Total on stock', 'Commited/Sold', 'Available']].sum().reset_index().copy()
# Colocar el código de empresa
gmb_inventory['COD_EMPRESA']='GMB'
# Renombrar columnas
gmb_inventory = gmb_inventory.rename(columns={'Stock Code (CU)': 'PRODUCTO',
                                              'Total on stock':'TotalOnStock',
                                              'Commited/Sold':'Committed',
                                              'Available':'Available'})
gmb_inventory = gmb_inventory[['COD_EMPRESA', 'PRODUCTO','TotalOnStock', 'Committed']]
print(gmb_inventory.shape)
gmb_inventory.head()

(2113, 9)
(2112, 9)
(1829, 4)


,COD_EMPRESA,PRODUCTO,TotalOnStock,Committed
0,GMB,105-8028,746.5,0.0
1,GMB,106-4001,0.0,0.0
2,GMB,106-8028,0.0,0.0
3,GMB,107-0001,503.1,3.2
4,GMB,107-0002,996.5,30.0


In [7]:
print('Total de metros en stock',  gmb_inventory['TotalOnStock'].sum())
print('Total commited',  gmb_inventory['Committed'].sum())

Total de metros en stock 310316.13
Total commited 136385.36


In [10]:
# Leer archivo inventario
slu_inventory = pd.read_excel('SLU.xlsx').rename(columns={'Atock Code':'Stock code'})
print(slu_inventory.shape)
# remover vacios
slu_inventory = slu_inventory[~slu_inventory['Stock code'].isna()]
# slu_inventory.head()
# Totalizar por Stockcode
slu_inventory = slu_inventory.groupby(['Stock code'])[['Stock', 'Committed', 'Available']].sum().reset_index().copy()
# Colocar el código de empresa
slu_inventory['COD_EMPRESA']='SLU'
print(slu_inventory.shape)
slu_inventory = slu_inventory.rename(columns={'Stock code':'PRODUCTO',
                                              'Stock':'TotalOnStock',
                                              'Committed':'Committed', 
                                              'Available':'Available'})
slu_inventory = slu_inventory[['COD_EMPRESA', 'PRODUCTO','TotalOnStock', 'Committed']]
print(slu_inventory.shape)
slu_inventory.head()

(2230, 9)
(1889, 5)
(1889, 4)


,COD_EMPRESA,PRODUCTO,TotalOnStock,Committed
0,SLU,78187,54.86,0.0
1,SLU,105-8028,0.00,0.0
2,SLU,106-4001,0.00,0.0
3,SLU,106-8028,0.00,0.0
4,SLU,107-0001,1584.00,38.0


In [12]:
# Leer archivo inventario
ukn_inventory = pd.read_excel('UK.xlsx')
print(ukn_inventory.shape)
# Seleccionar columnas requeridas
ukn_inventory = ukn_inventory[['Stock Code', 'Total Stock', 'Compromised']].copy()
#ukn_inventory.head()
#organizar
ukn_inventory = ukn_inventory.rename(columns={'Stock Code':'PRODUCTO',
                                              'Total Stock':'TotalOnStock',
                                              'Compromised':'Committed'})
# agrupar
ukn_inventory = ukn_inventory.groupby(['PRODUCTO'])[['TotalOnStock', 'Committed']].sum().reset_index()
# Colocar el código de empresa
ukn_inventory['COD_EMPRESA']='UK'

ukn_inventory = ukn_inventory[['COD_EMPRESA', 'PRODUCTO','TotalOnStock', 'Committed']]
print(ukn_inventory.shape)
ukn_inventory.head()

(677, 14)
(633, 4)


,COD_EMPRESA,PRODUCTO,TotalOnStock,Committed
0,UK,107-0001,28.4,0.0
1,UK,107-0002,273.9,10.0
2,UK,107-0020,123.3,0.0
3,UK,107-0033,65.3,0.0
4,UK,107-0034,250.6,56.0


In [13]:
# unir los tres reportes de inventario
inventory_on_hand = pd.concat([gmb_inventory, slu_inventory, ukn_inventory]).copy()
# Agregar el tipo de dato
inventory_on_hand['TipoDato'] = 'Initial_Inventory'
# Borrar los archivos anteriores
print(inventory_on_hand.shape)
inventory_on_hand.head()

(4351, 5)


,COD_EMPRESA,PRODUCTO,TotalOnStock,Committed,TipoDato
0,GMB,105-8028,746.5,0.0,Initial_Inventory
1,GMB,106-4001,0.0,0.0,Initial_Inventory
2,GMB,106-8028,0.0,0.0,Initial_Inventory
3,GMB,107-0001,503.1,3.2,Initial_Inventory
4,GMB,107-0002,996.5,30.0,Initial_Inventory


<h2>Leyendo status de planta</h2>

In [14]:
hoy = datetime(2022,4, 22).strftime("%Y%m%d")
print(f'Fecha de Hoy para trabajo {hoy}')
# ruta donde estan los archivos
path = f'Y:\\LISTADOS\\1. Reportes_consolidados\\Reportes_{hoy}.xlsx'
# Leer etapas
etapas_pqn = pd.read_excel(path, sheet_name='letapas', dtype={'CODPRO':object, 'COD_CLIENTE':object})
# tomar solo lo de Europa
etapas_pqn = etapas_pqn[etapas_pqn['COD_CLIENTE'].isin(pqn_customers)].copy()
print(etapas_pqn.shape)
# Sacar un resumen de las etapas en el archivo
summary_transits = etapas_pqn.groupby(['CLIENTE', 'PAIS' , 'FACTURA', 'FECHA DE FACTURA', 'TRANSPORTE', 'UN', 'CANTIDAD EXPORTACION'])[['CANTIDAD DESPACHADA']].sum().reset_index()
summary_transits.head()

Fecha de Hoy para trabajo 20220422
(1155, 66)


,CLIENTE,PAIS,FACTURA,FECHA DE FACTURA,TRANSPORTE,UN,CANTIDAD EXPORTACION,CANTIDAD DESPACHADA
0,"EURO-SPRADLING, S.L.U.",CO,47854,2022-03-14,VIA MARITIMA,MT,7864.5,7864.5
1,"EURO-SPRADLING, S.L.U.",CO,47855,2022-03-14,AEREO,MT,839.4,839.4
2,"EURO-SPRADLING, S.L.U.",CO,47885,2022-03-18,VIA MARITIMA,MT,8182.7,8182.7
3,"EURO-SPRADLING, S.L.U.",CO,47913,2022-03-24,VIA MARITIMA,MT,8198.7,8198.7
4,"EURO-SPRADLING, S.L.U.",CO,47950,2022-03-31,VIA MARITIMA,MT,8738.3,8738.3


In [15]:
# Leer pendientes
pendiente_pqn = pd.read_excel(path, sheet_name='pendiente', dtype={'COD':object, 'Cliente':object})
# Clasificar pendientes en clientes Spradling Eur y others
pendiente_pqn['Cliente']= pendiente_pqn['Cliente'].apply(lambda x: x if x in pqn_customers else 'OTHERS')
print(pendiente_pqn.shape)
pendiente_pqn.head()

(6096, 50)


,Pais,Mdo,Cliente,Nombre Cliente,Sc,Ciudad,Orden,Consecutivo,Nombre Producto,REFERENCIA,Tp C,Tpo P,Est,Desp.R.APS,F. Entrega.,Desp.RealR,F. Inicial,MES,ESTADO,Pendiente,Total,Und,PENDIENTE MTS,TOTAL MTS,OrdenP,Est.1,Descripción Est,CantidadProg,MER,Turno,Maquina,SigFecTurn,Existencia MTS,Separación,SEPARACIÓN MTS,ASIGNADO OLA-WMS,F.Ent.CEDI,Días en CEDI,MDO,Pais2,Precio Lista,Premium,Precio Factura,Mon,VALOR UN. USD,VALOR TOTAL PEND USD,STOCK,COD,CLIENTE ESP.,LastUpdate
0,CO,EX,OTHERS,ATTHENA EMBARCACOES S.A.,1,SAO PAULO,78872,397136,CHRONOS-A PORTOBELLO,CHRONOS-A,CP,PT,AC,2022-01-18,2022-01-25,2022-01-18,2022-01-25,2022-01,DELAYED 4+,510.0,510.0,MT,510.0,510.0,*,*,*,*,*,*,*,*,577.7,561.0,561.0,547.7,2022-01-19 00:00:00,-1,LAT,BR,13.87,0,13.87,US,13.87,7073.7,SIN,090032,*,20220422
1,CO,EX,OTHERS,ATTHENA EMBARCACOES S.A.,1,SAO PAULO,79747,398054,CHRONOS-A SIBERIA,CHRONOS-A,CP,PT,AC,2022-02-04,2022-02-11,2022-02-04,2022-02-11,2022-02,DELAYED 4+,510.0,510.0,MT,510.0,510.0,*,*,*,*,*,*,*,*,516.2,516.0,516.0,356.7,2022-01-19 00:00:00,16,LAT,BR,13.87,0,13.87,US,13.87,7073.7,SIN,090012,*,20220422
2,CO,EX,OTHERS,ATTHENA EMBARCACOES S.A.,1,SAO PAULO,79463,397582,DIAMANTE-A SUNSET,DIAMANTE-A,CP,PT,AC,2022-01-25,2022-02-01,2022-01-25,2022-02-01,2022-01,DELAYED 4+,810.0,810.0,MT,810.0,810.0,257877,T,EN TURNO GENERA,1246,E,167,RAMA III,2022-05-11 00:00:00,1082.9,730.0,730.0,730.1,2022-01-10 00:00:00,15,LAT,BR,12.63,0,12.63,US,12.63,10230.3,SIN,084299,*,20220422
3,CO,EX,OTHERS,ATTHENA EMBARCACOES S.A.,1,SAO PAULO,78872,397136,DIAMANTE-A TONIC,DIAMANTE-A,CP,PT,AC,2022-01-18,2022-01-25,2022-01-18,2022-01-25,2022-01,DELAYED 4+,420.0,420.0,MT,420.0,420.0,257878,T,EN TURNO GENERA,2491,E,170,RAMA III,2022-05-11 00:00:00,356.3,356.0,356.0,90.5,2022-01-07 00:00:00,11,LAT,BR,12.63,0,12.63,US,12.63,5304.6,SIN,089664,*,20220422
4,CO,EX,OTHERS,ATTHENA EMBARCACOES S.A.,1,SAO PAULO,78872,397136,MOSAIQUE-A HUMUS,MOSAIQUE-A,CP,PT,AC,2022-01-18,2022-01-25,2022-01-18,2022-01-25,2022-01,DELAYED 4+,510.0,510.0,MT,510.0,510.0,*,*,*,*,*,*,*,*,560.7,561.0,561.0,560.7,2022-01-09 00:00:00,9,LAT,BR,9.31,0,9.31,US,9.31,4748.1,SIN,089951,*,20220422


In [16]:
# Revisar existencias
existencias_pqn =  pd.read_excel(path, sheet_name='existencias', dtype={'Produc':object})

In [17]:
# listado de lstocks
lstocks = pd.read_excel(path, sheet_name='lstock', dtype={'Stock':object, 'Producto':object, 'Client':object})
# Fitrar por los de Europa
lstocks = lstocks[(lstocks['Act']=='AC') & (lstocks['Client'].isin(pqn_customers))]
# armar pivots
lstocks = lstocks.pivot_table(index=['Stock', 'Producto'], columns=['Client'], values=['Act'], aggfunc=sum).reset_index().fillna("CN")
# Arreglar nombres de columnas
lstocks.columns = [x[0] if x[1] == '' else x[1] for x in lstocks.columns]


In [18]:
# Niveles de iventario en Plantas
niveles_pqn = pd.read_excel(path, sheet_name='lniveles', dtype={'CODIGO':object})
# Preparar para eliminar productos in nivel
niveles_pqn['Total'] = niveles_pqn['SAFETY_STOCK'] + niveles_pqn['REORDER_POINT'] + niveles_pqn['AVG_DEM_12_MONTHS']
# Colocar política en PQN
niveles_pqn['Politica_PQN'] = niveles_pqn['Total'].apply(lambda x: 'MTS' if x > 0 else 'MTO')
# unir con reporte de Lstocks

lstocks = pd.merge(left=lstocks, 
                   right=niveles_pqn[['CODIGO', 'Politica_PQN']], 
                   left_on=['Producto'], 
                   right_on=['CODIGO'], 
                   how='left')[['Stock', 'Producto', 'Politica_PQN'] + pqn_customers].copy()


In [19]:
# Lsabana para la programaciòn de turnos
lsabana = pd.read_excel(path, sheet_name='lsabana', dtype={'Referencia':object, 'Producto':object, 'Client':object})
# renombrar columnas
lsabana.rename(columns={'Mts.Pend.':'TotalProgramado'}, inplace=True)
# Totalizar órdenes programadas
lsabana = lsabana.groupby(['Referencia'])[['TotalProgramado']].sum().reset_index()

In [20]:
# Lrollo para conocer el proceso
lrollo = pd.read_excel(path, sheet_name='lrollo', dtype={'CODIGO':object, 'Producto':object, 'Client':object})
# Construir PivotTable
lrollo = lrollo.pivot_table(values=['METROS'], index=['CODIGO'], columns=['ESTADO'], aggfunc=sum).fillna(0.0).reset_index()
# arreglar título
lrollo.columns = [x[0] if x[1] == '' else ' '.join(list(x)) for x in lrollo.columns]

In [21]:
# Planta asignada
lplanta = pd.read_excel(path, sheet_name='lproducto_sc', dtype={'Produc':object})
lplanta.head()

,Produc,E,PL,LastUpdate
0,091092,I,CO,20220422
1,091632,I,CO,20220422
2,087786,I,CO,20220422
3,073071,I,CO,20220422
4,064874,I,CO,20220422


In [22]:
# maestro de productos PQN
lproductos = pd.read_excel(path, sheet_name='lproducto_tipo_sc', dtype={'Produc':object, 'Refer.':object, 'Base':object})
lproductos.head()

,Produc,Nombre Comercial,Nombre Tecnico,Clasif,Estado,Tipo P,Origen,Nombre Origen Categorizacion,SegMer,Nombre Segmento Mercado,Nicho,Nombre Nicho,Aplicac.,Nombre Aplicacion,Gama,Nombre Gama,Refer.,Nombre Referencia,Base,Nombre Base,F.Ingreso,Mercado,LastUpdate
0,091092,.,.,DS,I,ED,NE,NMI ESPECIALES,IYR,INSTITUCIONAL Y RESIDENCIAL,NPND,NICHO PENDIENTE,APEND,APLICACION PENDIENTE,SUP,SUPERIOR,651731,1,NaN,NaN,2016-12-30,EXPORTACIONES,20220422
1,091632,..........................,..........................,DS,I,ED,NE,NMI ESPECIALES,IYR,INSTITUCIONAL Y RESIDENCIAL,NPND,NICHO PENDIENTE,PISOS,PISO,SUP,SUPERIOR,085229,PISO NAPOLES,NaN,NaN,2017-08-03,NACIONAL,20220422
2,087786,. BASE TAPIK CELFIL 120,. BASE TAPIK CELFIL 120,DS,I,ED,PQ,PQN,SMP,SEGMENTO MERCADO PENDIENTE,NPND,NICHO PENDIENTE,APEND,APLICACION PENDIENTE,GMP,GAMA PENDIENTE,084840,TAPIK CELFIL 120,NaN,NaN,2013-08-13,NACIONAL,20220422
3,076341,CANALETA AK2-GA69 60X90 MM TELEMECANIQUE EXP,CANALETA AK2-GA69 60X90 MM TELEMECANIQUE EXP,CP,A,FA,OP,ORIGEN - PENDIENTE,SMP,SEGMENTO MERCADO PENDIENTE,NPND,NICHO PENDIENTE,APEND,APLICACION PENDIENTE,GMP,GAMA PENDIENTE,999999,OTROS,NaN,NaN,2004-01-14,EXPORTACIONES,20220422
4,076343,CONTACTOR SIEMENS 3RT1045 2NA+2NC 220 V EXP,CONTACTOR SIEMENS 3RT1045 2NA+2NC 220 V EXP,CP,A,FA,OP,ORIGEN - PENDIENTE,SMP,SEGMENTO MERCADO PENDIENTE,NPND,NICHO PENDIENTE,APEND,APLICACION PENDIENTE,GMP,GAMA PENDIENTE,999991,NO PRIMERAS,NaN,NaN,2004-01-14,AMBOS,20220422


<h2>Usar Ventas Mensuales</h2>

In [23]:
# Cargar ventas históricas desde archivo parquet
ventas_df = pd.read_parquet(f'Y:\\PlaneacionAbastecimiento\\data\\input\\ventas\\spradling_eur_{hoy[0:6]}.parquet')
print(ventas_df.shape)
# Eliminar ventas a asociados
ventas_df = ventas_df[ventas_df['I_ASOCIADO']=='N']
# Filtrar solo el periodo de interés
ventas_df = ventas_df[ventas_df['PERIODO'].isin(['202203'])]
print(ventas_df.shape)
#Agrupar ventas por periodo y sku
ventas_df = ventas_df.groupby(['PERIODO', 'NOM_EMPRESA', 'D_CODIGO_ALTERNO'])[['Q_CANTIDAD_UOM_UNICA']].sum().reset_index()
ventas_df.head()

(73634, 29)
(5238, 29)


,PERIODO,NOM_EMPRESA,D_CODIGO_ALTERNO,Q_CANTIDAD_UOM_UNICA
0,202203,EURO SPRADLING INTERNATIONAL,078187,27.71
1,202203,EURO SPRADLING INTERNATIONAL,107-0001,405.90
2,202203,EURO SPRADLING INTERNATIONAL,107-0002,290.70
3,202203,EURO SPRADLING INTERNATIONAL,107-0020,655.10
4,202203,EURO SPRADLING INTERNATIONAL,107-0033,258.10


In [24]:
# Ventas actuales
with open('../../../keys.json') as file:
    keys = json.load(file)

cad_con= f'{keys["global_user_noexpirer_user"]}/{keys["global_user_noexpirer_pass"]}@172.16.1.3/global'
cad_con

'Interfaz/global2020@172.16.1.3/global'

In [25]:
def descargar_data(sql:str, periods:list)->pd.DataFrame:
    list_files = []    
    for periodo in periods:  
        with cx_Oracle.connect(cad_con) as con:
            list_files.append(pd.read_sql(sql=sql.format(periodo), con=con))
    return pd.concat(list_files) 

In [26]:
sql_query = '''
SELECT * FROM GLOBAL.V_VENTAS_LOCAL_01 WHERE Periodo = {0} AND COD_EMPRESA IN ('023','025','028')
'''
ventas_actuales = descargar_data(sql=sql_query, periods=[hoy[0:6]])
# ventas actuales
ventas_actuales = ventas_actuales[ventas_actuales['I_ASOCIADO']=='N'].copy()
# Agrupar ventas actuales
ventas_actuales = ventas_actuales.groupby(['COD_EMPRESA', 'PERIODO', 'D_CODIGO_ALTERNO'])[['Q_CANTIDAD_UOM_UNICA']].sum().reset_index()

In [27]:
ventas_actuales.head()

,COD_EMPRESA,PERIODO,D_CODIGO_ALTERNO,Q_CANTIDAD_UOM_UNICA
0,023,202204,107-0001,157.60
1,023,202204,107-0002,527.63
2,023,202204,107-0020,456.70
3,023,202204,107-0033,21.50
4,023,202204,107-0034,658.10


<h2>Generar reporte de Productos que no cumplen lote mínimo</h2>

<p>La siugiente función nos permitira determinar qué productos se pondrán en este reporte</p>

In [28]:
def comentar_pedido(pending:float, in_production:float, existencias:float)->str:
    if pending > 0 and in_production + existencias == 0.0:
        if pending >= 500:
            return 'Push production scheduling'
        else:
            return 'Cancel Purchase order?'
    else:
        return ''

In [29]:
# filtrar solo por las ordenes de Europa
order_analysis_df = pendiente_pqn[pendiente_pqn['Cliente'].isin(pqn_customers)]
# Crear un dataframe donde uniremos la información
order_analysis_df = order_analysis_df.pivot_table(values=['Pendiente', 'Orden'], 
                                                  index=['STOCK', 'Nombre Producto'], 
                                                  columns=['Nombre Cliente'], aggfunc=sum).fillna(0.0)
print('Crear un dataframe donde uniremos la información',order_analysis_df.shape)
# Calcular el total pendiente
order_analysis_df['TotalPending'] = [sum(order_analysis_df.iloc[x,:]) for x in range(order_analysis_df.shape[0])]
# resetar Index
order_analysis_df = order_analysis_df.reset_index()
# Eliminar el segundo orden de las columnas
order_analysis_df.columns = [x[0] if x[1] == '' else ' '.join(list(x)) for x in order_analysis_df.columns]
print('Eliminar el segundo orden de las columnas',order_analysis_df.shape)
# incluir los metros planeados en producción
order_analysis_df = pd.merge(left=order_analysis_df,
                             right=lsabana,
                             how='left',
                             left_on='STOCK',
                             right_on='Referencia')

print('incluir los metros planeados en producción',order_analysis_df.shape)
# incluir los metros en corriendo
order_analysis_df = pd.merge(left=order_analysis_df,
                             right=lrollo,
                             how='left',
                             left_on='STOCK',
                             right_on='CODIGO')

print('incluir los metros en corriendo', order_analysis_df.shape)
# incluir existencias
order_analysis_df = pd.merge(left=order_analysis_df,
                             right=existencias_pqn,
                             how='left',
                             left_on='STOCK',
                             right_on='Produc')

print('incluir existencias', order_analysis_df.shape)
# eliminar columnas innecesarias
order_analysis_df.drop(columns=['Referencia', 'CODIGO', 'Produc'], inplace=True)
print('eliminar columnas innecesarias', order_analysis_df.shape)
# eliminar nulos
order_analysis_df = order_analysis_df.fillna(0.0)
print('eliminar nulos', order_analysis_df.shape)
# Generar criterio para remover filas
order_analysis_df['Total_En_Produccion'] = order_analysis_df['TotalProgramado'] + order_analysis_df['METROS EN PROCESO'] + order_analysis_df['METROS POR REVISAR']
print('Generar criterio para remover filas' ,order_analysis_df.shape)
# generar comentarios
order_analysis_df['Comments'] = order_analysis_df.apply(lambda x: comentar_pedido(pending=x['TotalPending'], 
                                                                                  in_production=x['Total_En_Produccion'], 
                                                                                 existencias=x['Existencias Mts']), 
                                                        axis=1)
print('generar comentarios', order_analysis_df.shape)
# Extraer solo aquellas cancelables
order_analysis_df = order_analysis_df[order_analysis_df['Comments']=='Cancel Purchase order?'].copy()
print('Extraer solo aquellas cancelables', order_analysis_df.shape)
# Mostrar el reporte
order_analysis_df.head()

Crear un dataframe donde uniremos la información (401, 3)
Eliminar el segundo orden de las columnas (401, 6)
incluir los metros planeados en producción (401, 8)
incluir los metros en corriendo (401, 11)
incluir existencias (401, 17)
eliminar columnas innecesarias (401, 14)
eliminar nulos (401, 14)
Generar criterio para remover filas (401, 15)
generar comentarios (401, 16)
Extraer solo aquellas cancelables (94, 16)


,STOCK,Nombre Producto,"Pendiente EURO-SPRADLING, S.L.U.",Pendiente SPRADLING INTERNATIONAL GMBH,Pendiente SPRADLING UK LTD.,TotalPending,TotalProgramado,METROS EN PROCESO,METROS POR REVISAR,PLANTA,MDO,Existencias Mts,Retenido,LastUpdate,Total_En_Produccion,Comments
6,107-1001,VALENCIA-A CHAMPAGNE,238.9,210.0,0.0,448.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cancel Purchase order?
11,107-2001,VALENCIA-A MERLOT,0.0,150.0,0.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cancel Purchase order?
13,107-2020,VALENCIA-A SONNE,0.0,150.0,30.0,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cancel Purchase order?
19,107-2107,VALENCIA-A SKYLIGHT,270.0,120.0,60.5,450.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cancel Purchase order?
26,107-2114,VALENCIA-A ROSE,300.0,150.0,0.0,450.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cancel Purchase order?


In [30]:
# Listar las ordenes abiertas para los productos identificados
temp = pendiente_pqn[pendiente_pqn['STOCK'].isin(order_analysis_df['STOCK'])][['STOCK','Orden']]
ordenes = dict()

for i in range(temp.shape[0]):
    if not temp.iloc[i][0] in ordenes.keys():
        ordenes[temp.iloc[i][0]]=list()
    ordenes[temp.iloc[i][0]].append(temp.iloc[i][1])


In [31]:
order_analysis_df['Open Orders'] = order_analysis_df['STOCK'].map(ordenes)

<p>Cargar y procesar el forecast</p>

In [32]:
# Generar días del mes
def dias_mes(x:str):
    x = str(x)
    año = int(x[0:4])
    mes = int(x[4:])
    dias = calendar.monthrange(año, mes)[1]
    # print(x, año, mes, dias)
    return dias

# leer forecast
forecast_df = pd.read_excel('Y:\\PlaneacionDemanda\\Rollovers\\Current\\temporal\\ForecastParaSuggested.xlsx')
# Aplicar un unpivot para trabajar con el RMSE
id_vars = ['COD_EMPRESA', 'PRODUCTO', 'RMSE']
# aplanar titulos de columnas
value_vars = list(forecast_df.columns)
value_vars = [x for x in forecast_df.columns if x not in id_vars]
forecast_df = pd.melt(frame = forecast_df, 
                      id_vars= id_vars, 
                      value_vars= value_vars).rename(columns={'variable': 'PERIODO', 
                                                              'value':'UNIDADES_MENSUALES', 
                                                              'RMSE':'RMSE_MENSUAL'})


In [33]:
# Calcular los días del mes
forecast_df['DIAS_MES'] = forecast_df['PERIODO'].apply(dias_mes)
# Calcular las ventas pronosticadas diárias
forecast_df['UNIDADES_DIARIAS'] = forecast_df['UNIDADES_MENSUALES']/forecast_df['DIAS_MES']

In [34]:
# Obtener un dataframe de periodos
periodos_df = pd.read_excel('Y:\\PlaneacionAbastecimiento\\data\\masters\\Dim_Date_Fechas.xlsx')
# Organizar
periodos_df = periodos_df[['Periodo', 'Date', 'WeekYear']].rename(columns={'Date':'Fecha', 'Periodo':'PERIODO', 'WeekYear':'Semana'})
# crear la variable de Semana
periodos_df['Semana'] = periodos_df['Fecha'].dt.year*100+periodos_df['Semana']
# Omitir datos posteriores a hoy
periodos_df = periodos_df[periodos_df['Fecha']>=hoy_date]

print(periodos_df.shape)
periodos_df.head()

(1348, 3)


,PERIODO,Fecha,Semana
2670,202204,2022-04-24,202218
2671,202204,2022-04-25,202218
2672,202204,2022-04-26,202218
2673,202204,2022-04-27,202218
2674,202204,2022-04-28,202218


In [35]:
# unir forecast con el calendario en la variable periodos_df
forecast_df = pd.merge(left=forecast_df, right=periodos_df, left_on=['PERIODO'], right_on=['PERIODO'], how='inner').copy()
# Calcular RMSE Semanal
forecast_df['RMSE_SEMANAL'] = forecast_df['RMSE_MENSUAL']*(7**0.5)/(30**0.5)
# Reagrupar la serie de datos en semanas
forecast_df = forecast_df.groupby(['COD_EMPRESA', 'PRODUCTO', 'Semana', 'RMSE_SEMANAL'])[['UNIDADES_DIARIAS']].sum().reset_index().rename(columns={'UNIDADES_DIARIAS': 'UNIDADES_SEMANALES'})
# redondear
forecast_df['UNIDADES_SEMANALES'] = forecast_df['UNIDADES_SEMANALES'].apply(round)
print('forecast_df', forecast_df.shape)
forecast_df.head()

forecast_df (246862, 5)


,COD_EMPRESA,PRODUCTO,Semana,RMSE_SEMANAL,UNIDADES_SEMANALES
0,GMB,105-8028,202218,87.532746,141
1,GMB,105-8028,202219,87.532746,19
2,GMB,105-8028,202220,87.532746,19
3,GMB,105-8028,202221,87.532746,19
4,GMB,105-8028,202222,87.532746,19


<h2>Escribir archivo</h2>

In [36]:
with pd.ExcelWriter('información.xlsx') as writer:
    summary_transits.to_excel(writer, sheet_name='Transits_summary', index=False)
    etapas_pqn.to_excel(writer, sheet_name='Transits_to_EUR', index=False)
    ventas_df.to_excel(writer, sheet_name='ventas_mensuales', index=False)    
    inventory_on_hand.to_excel(writer, sheet_name='EUR_Inventory', index=False)
    pendiente_pqn.to_excel(writer, sheet_name='PQN_Pendings', index=False)
    existencias_pqn.to_excel(writer, sheet_name='PQN_Inventory', index=False)
    lstocks.to_excel(writer, sheet_name='Lstocks', index=False)
    forecast_df.to_excel(writer, sheet_name='forecast', index=False)
    order_analysis_df.to_excel(writer, sheet_name='CancelableOrders', index=False)
    ventas_actuales.to_excel(writer, sheet_name=f'Ventas_a_{hoy}', index=False)
    lplanta.to_excel(writer, sheet_name='AsignacionPlanta', index=False)
    lproductos.to_excel(writer, sheet_name='Lproducto', index=False)
    periodos_df.to_excel(writer,sheet_name='dim_calendar', index=False)

<h2>Ejecutar el Modelo</h2>

<h2>Procesar arrivals</h2>
<p>Los arrivals estan conformados por dos partes:</p>
<ol>
    <li><strong>Los tránsitos</strong> que han sido confirmados por cada bodega y que tienen una fecha estimada (EAT) estimable;</li>
    <li><strong>Los pedidos pendientes</strong> actuales que se asume se entregan 31 días después. Sin embargo, este supuesto debe mejorarse dado que habrán productos que estan en inventario, por lo que pueden ser despachados durante la semana actual y, los productos que estan en proceso que podrán despacharse en la siguiente semana, y los que estan programados que básicamente se puede estimar su despacho de acuerdo con la fecha de programación y su complejidad.</li>
</ol>
<p>Resulta necesario que el punto anterior sea ajustado para tener mejores proyecciones de inventario.</p>

In [38]:
# Exprotaciones comfirmadas en tránsito
gmb_exports = [2954,2953,2980,47909,3018,47949,3050,3086,47984,3098,3101]
slu_exports = [2940,2971,47913,3000,3003,47950,3035,3036,47986,3074,3075,48018]
ukn_exports = [47952, 47953,47985,3066]

In [39]:
model_periods = sorted(list(forecast_df['Semana'].unique()))
print(model_periods[0], model_periods[-1])

202218 202409


In [40]:
# Tomar una copia de etapas
arrivals_df = etapas_pqn.copy()

# Remover tránsitos que no hayan sido reportados
arrivals_df = arrivals_df[arrivals_df['FACTURA'].isin(gmb_exports+slu_exports+ukn_exports)].copy()

# remplazar cliente en arrivals
arrivals_df['COD_BODEGA'] = arrivals_df['COD_CLIENTE'].map({'101111':'SLU', '100459':'GMB', '101724':'UK'})

# Dejar solo columnas relevantes
arrivals_df = arrivals_df[['PAIS','COD_BODEGA', 'FACTURA', 'STOCK', 'CANTIDAD DESPACHADA', 'UN',
                           'DESPACHO REAL PQN', 'TRANSPORTE', 'FECHA DE FACTURA', 'LastUpdate' ]].copy()

# calcular fecha de llegada con base en la fecha de la factura
arrivals_df['EAT'] = arrivals_df.apply(lambda x: x['FECHA DE FACTURA'] + timedelta(days=7) if x['TRANSPORTE']=='AEREO' else x['FECHA DE FACTURA'] + timedelta(days=31), axis=1)  


# Unir arrivals con periodos según la fecha de llegada para poder calcular arrivals por semana
arrivals_df = pd.merge(left=arrivals_df,
                       right=periodos_df[['Fecha', 'Semana']],
                       left_on=['EAT'],
                       right_on=['Fecha'],
                       how='left')

# Alistar para concatenar
arrivals_df = arrivals_df.groupby(['COD_BODEGA', 'STOCK', 'Semana'])[['CANTIDAD DESPACHADA']].sum().reset_index()

# renombrar
arrivals_df.rename(columns={'CANTIDAD DESPACHADA': 'Cantidad'}, inplace=True)

print('Arrivals:', arrivals_df.shape)
arrivals_df.head()

Arrivals: (508, 4)


,COD_BODEGA,STOCK,Semana,Cantidad
0,GMB,105-8028,202220.0,1388.8
1,GMB,107-0001,202221.0,535.5
2,GMB,107-0020,202220.0,283.2
3,GMB,107-0033,202218.0,210.0
4,GMB,107-0033,202221.0,97.2


In [41]:
# tomar una copia de los pedidos pendientes
pending_df = pendiente_pqn[['Pais', 'Cliente', 'Desp.R.APS', 'PENDIENTE MTS', 'STOCK', 'LastUpdate']].copy()

# remplazar cliente en arrivals
pending_df['COD_BODEGA'] = pending_df['Cliente'].map({'101111':'SLU', '100459':'GMB', '101724':'UK'})

# dejar solo clientes de Europa
pending_df = pending_df[~pending_df['COD_BODEGA'].isna()]

# calcular el EAT ESte no tiene en cuenta si el producto está en inventario, esta en proceso o progrmado 
# y debería tenerse en cuenta
pending_df['EAT'] = pending_df.apply(lambda x: max(x['Desp.R.APS'], hoy_date) + timedelta(days=31) ,axis=1)

# unir con dimensión de fechas para extraer la semana
pending_df = pd.merge(left=pending_df,
                       right=periodos_df[['Fecha', 'Semana']],
                       left_on=['EAT'],
                       right_on=['Fecha'],
                       how='left')

# Alistar para concatenar
pending_df = pending_df.groupby(['COD_BODEGA', 'STOCK', 'Semana'])[['PENDIENTE MTS']].sum().reset_index()

# renombrar
pending_df.rename(columns={'PENDIENTE MTS': 'Cantidad'}, inplace=True)

print('Arrivals:', arrivals_df.shape)
pending_df.head()

Arrivals: (508, 4)


,COD_BODEGA,STOCK,Semana,Cantidad
0,GMB,107-0001,202222,52.5
1,GMB,107-0001,202226,180.0
2,GMB,107-0001,202229,270.0
3,GMB,107-0002,202229,840.0
4,GMB,107-0020,202222,750.0


<p>Para finalizar, los arrival resultantes tendrán que concatenar los tránsitos y los pendientes que hemos calculado anteriormente, y finalmente alistarlos para su uso con el modelo</p>

In [42]:
# concatentar los arrivals de transitos y de pendings
arrivals = pd.concat([arrivals_df, pending_df]).copy()
print('Arrivals resultante:', arrivals.shape)

# agrupar eliminando duplicados
arrivals = arrivals.groupby(['COD_BODEGA', 'STOCK', 'Semana'])[['Cantidad']].sum().reset_index()

arrivals.head()

Arrivals resultante: (1725, 4)


,COD_BODEGA,STOCK,Semana,Cantidad
0,GMB,105-8028,202220.0,1388.8
1,GMB,107-0001,202221.0,535.5
2,GMB,107-0001,202222.0,52.5
3,GMB,107-0001,202226.0,180.0
4,GMB,107-0001,202229.0,270.0


In [43]:
# pivotear arrivals con la semana. Revisaremos qué periodos faltan para completar el array
arrivals = arrivals.pivot_table(index=['COD_BODEGA', 'STOCK'], columns=['Semana'], values=['Cantidad'], aggfunc=np.sum)
# Corregir titulos
arrivals.columns = [x[1] for x in arrivals.columns]

In [44]:
# completar las columnas faltantes
for columna in model_periods:
    if columna not in arrivals.columns:
        arrivals[columna] = 0.0
# reordenar las columnas
arrivals = arrivals[model_periods]

# eliminar na
arrivals.fillna(0.0, inplace=True)

print(arrivals.shape)
arrivals.head()

(861, 98)


202218.0  202219.0  202220.0  202221.0  202222.0  \
COD_BODEGA STOCK                                                        
GMB        105-8028       0.0       0.0    1388.8       0.0       0.0   
           107-0001       0.0       0.0       0.0     535.5      52.5   
           107-0002       0.0       0.0       0.0       0.0       0.0   
           107-0020       0.0       0.0     283.2       0.0     750.0   
           107-0033     210.0       0.0       0.0      97.2       0.0   

                     202223.0  202224.0  202225.0  202226.0  202227.0  \
COD_BODEGA STOCK                                                        
GMB        105-8028       0.0       0.0       0.0       0.0       0.0   
           107-0001       0.0       0.0       0.0     180.0       0.0   
           107-0002       0.0       0.0       0.0       0.0       0.0   
           107-0020       0.0       0.0       0.0     600.0       0.0   
           107-0033       0.0       0.0       0.0     180.0       0.0   

                     202228.0  202229.0  202230.0  202231.0  202232.0  \
COD_BODEGA STOCK                                                        
GMB        105-8028       0.0       0.0       0.0       0.0       0.0   
           107-0001       0.0     270.0       0.0       0.0       0.0   
           107-0002       0.0     840.0       0.0       0.0       0.0   
           107-0020       0.0     540.0       0.0       0.0       0.0   
           107-0033       0.0       0.0       0.0       0.0       0.0   

                     202233.0  202234.0  202235.0  202236.0  202237.0  \
COD_BODEGA STOCK                                                        
GMB        105-8028       0.0       0.0       0.0       0.0       0.0   
           107-0001       0.0       0.0       0.0       0.0       0.0   
           107-0002       0.0       0.0       0.0       0.0       0.0   
           107-0020       0.0       0.0       0.0       0.0       0.0   
           107-0033       0.0       0.0       0.0       0.0       0.0   

                     202238.0  202239.0  202240.0  202241.0  202242.0  \
COD_BODEGA STOCK                                                        
GMB        105-8028       0.0       0.0       0.0       0.0       0.0   
           107-0001       0.0       0.0       0.0       0.0       0.0   
           107-0002       0.0       0.0       0.0       0.0       0.0   
           107-0020       0.0       0.0       0.0       0.0       0.0   
           107-0033       0.0       0.0       0.0       0.0       0.0   

                     202243.0  202244.0  202245.0  202246.0  202247.0  \
COD_BODEGA STOCK                                                        
GMB        105-8028       0.0       0.0       0.0       0.0       0.0   
           107-0001       0.0       0.0       0.0       0.0       0.0   
           107-0002       0.0       0.0       0.0       0.0       0.0   
           107-0020       0.0       0.0       0.0       0.0       0.0   
           107-0033       0.0       0.0       0.0       0.0       0.0   

                     202248.0  202249.0  202250.0  202251.0  202252.0  \
COD_BODEGA STOCK                                                        
GMB        105-8028       0.0       0.0       0.0       0.0       0.0   
           107-0001       0.0       0.0       0.0       0.0       0.0   
           107-0002       0.0       0.0       0.0       0.0       0.0   
           107-0020       0.0       0.0       0.0       0.0       0.0   
           107-0033       0.0       0.0       0.0       0.0       0.0   

                     202253.0  202301.0  202302.0  202303.0  202304.0  \
COD_BODEGA STOCK                                                        
GMB        105-8028       0.0       0.0       0.0       0.0       0.0   
           107-0001       0.0       0.0       0.0       0.0       0.0   
           107-0002       0.0       0.0       0.0       0.0       0.0   
           107-0020       0.0       0.0       0.0       0.0       0.0   

<h3>Cargar el portafolio</h3>

In [45]:
# Leer el protafolio del Suggested
portfolio_df = pd.read_excel('C:\\Users\\pinilla-luis\\Desktop\\CleanPortafolioEuropa.xlsx', dtype={'CODE':object})
# Llenar los vacios
portfolio_df['Inventory'] = portfolio_df['Inventory'].fillna('NO')
# renombrar columnas
portfolio_df.rename(columns={'STOCK': 'Stock', '023': 'SLU', '025':'GMB', '028':'UK'}, inplace=True)
# organizar
portfolio_df = portfolio_df.melt(id_vars=['Stock', 'Inventory'], var_name='COD_EMPRESA', 
                                 value_vars=['GMB', 'UK', 'SLU'], value_name='SuggestedInvPolicy')
print(portfolio_df.shape)
portfolio_df.head()

c:\users\pinilla-luis\appdata\local\programs\python\python39\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(4386, 4)


,Stock,Inventory,COD_EMPRESA,SuggestedInvPolicy
0,TEJ-3104,NO,GMB,DSC
1,TEJ-2002,NO,GMB,DSC
2,TEJ-8001,NO,GMB,DSC
3,105-8028,NO,GMB,MTS
4,106-4001,NO,GMB,DSC


In [46]:
forecast = forecast_df.pivot_table(values=['UNIDADES_SEMANALES'],aggfunc=sum, 
                                      index=['COD_EMPRESA', 'PRODUCTO', "RMSE_SEMANAL"],
                                      columns=['Semana'])

# Arreglar títulos de columnas
forecast.columns = [x[1] for x in forecast.columns]

In [47]:
forecast = forecast.reset_index()

In [48]:
# Sacar RMSE
rmse = forecast[['COD_EMPRESA', 'PRODUCTO', 'RMSE_SEMANAL']].copy()

# Dejar forecast
forecast = forecast.drop(columns='RMSE_SEMANAL').set_index(['COD_EMPRESA', 'PRODUCTO']).copy()

In [49]:
print(forecast.shape)
forecast.head()

(2519, 98)


202218  202219  202220  202221  202222  202223  202224  \
COD_EMPRESA PRODUCTO                                                           
GMB         105-8028     141      19      19      19      19      75     116   
            107-0001      75      57      57      57      57      59      61   
            107-0002      66      86      86      86      86      80      76   
            107-0020     108     132     132     132     132     120     111   
            107-0033      29      32      32      32      32      31      31   

                      202225  202226  202227  202228  202229  202230  202231  \
COD_EMPRESA PRODUCTO                                                           
GMB         105-8028     116     116     108      88      88      88      88   
            107-0001      61      61      57      47      47      47      47   
            107-0002      76      76      72      62      62      62      62   
            107-0020     111     111     107      96      96      96      96   
            107-0033      31      31      28      22      22      22      22   

                      202232  202233  202234  202235  202236  202237  202238  \
COD_EMPRESA PRODUCTO                                                           
GMB         105-8028     121     127     127     127      72       0       0   
            107-0001      48      48      48      48      49      51      51   
            107-0002      60      60      60      60      67      75      75   
            107-0020     101     102     102     102      96      89      89   
            107-0033      26      27      27      27      27      27      27   

                      202239  202240  202241  202242  202243  202244  202245  \
COD_EMPRESA PRODUCTO                                                           
GMB         105-8028       0      17     118     118     118     118      84   
            107-0001      51      50      46      46      46      46      53   
            107-0002      75      72      56      56      56      56      61   
            107-0020      89      89      92      92      92      92      92   
            107-0033      27      27      25      25      25      25      28   

                      202246  202247  202248  202249  202250  202251  202252  \
COD_EMPRESA PRODUCTO                                                           
GMB         105-8028      71      71      71      41       0       0       0   
            107-0001      56      56      56      53      49      49      49   
            107-0002      63      63      63      63      64      64      64   
            107-0020      92      92      92      96     100     100     100   
            107-0033      29      29      29      27      25      25      25   

                      202253  202301  202302  202303  202304  202305  202306  \
COD_EMPRESA PRODUCTO                                                           
GMB         105-8028       0     139     139     139     139      80      35   
            107-0001      49      47      47      47      47      51      54   
            107-0002      64      67      67      67      67      56      47   
            107-0020     100      83      83      83      83      82      82   
            107-0033      25      26      26      26      26      23      22   

                      202307  202308  202309  202310  202311  202312  202313  \
COD_EMPRESA PRODUCTO                                                           
GMB         105-8028      35      35      55      70      70      70      75   
            107-0001      54      54      49      45      45      45      46   
            107-0002      47      47      79     103     103     103      95   
            107-0020      82      82      75      70      70      70      69   
            107-0033      22      22      22      22      22      22      22   

                      202314  202315  202316  202317  202318  202319  202320  \
COD_EMPRESA PRO

<p>Calcular los parámetros de inventario para un modelo periódico</p>

In [50]:
model_df = pd.merge(left=portfolio_df[['COD_EMPRESA', 'Stock', 'Inventory', 'SuggestedInvPolicy']],
                    left_on=['Stock'], 
                    how='inner', 
                    right=lstocks[['Stock', 'Politica_PQN']], right_on=['Stock'])
print(model_df.shape)
model_df.head()

(2295, 5)


,COD_EMPRESA,Stock,Inventory,SuggestedInvPolicy,Politica_PQN
0,GMB,105-8028,NO,MTS,MTO
1,UK,105-8028,NO,DSC,MTO
2,SLU,105-8028,NO,DSC,MTO
3,GMB,940-9001,SI,MTS,MTO
4,UK,940-9001,SI,DSC,MTO


In [51]:
# anexar los inventarios iniciales
model_df = pd.merge(left=model_df, 
                    right=inventory_on_hand[['COD_EMPRESA', 'PRODUCTO', 'TotalOnStock']], 
                    left_on=['COD_EMPRESA', 'Stock'], 
                    right_on=['COD_EMPRESA', 'PRODUCTO'], how='left').drop(columns=['Stock'])
print(model_df.shape)
model_df.head()

(2295, 6)


,COD_EMPRESA,Inventory,SuggestedInvPolicy,Politica_PQN,PRODUCTO,TotalOnStock
0,GMB,NO,MTS,MTO,105-8028,746.5
1,UK,NO,DSC,MTO,NaN,NaN
2,SLU,NO,DSC,MTO,105-8028,0.0
3,GMB,SI,MTS,MTO,940-9001,609.3
4,UK,SI,DSC,MTO,NaN,NaN


In [52]:
# Colocar RMSE
model_df = pd.merge(left=model_df, left_on=['COD_EMPRESA', 'PRODUCTO'],
                    how='left',         
                    right=rmse, right_on=['COD_EMPRESA', 'PRODUCTO'])

In [53]:
# Solamente calcularemos pedidos para aquellos productos que tienen Inventory en "SI"
model_df = model_df[model_df['Inventory']=='SI'].copy()

In [54]:
# asignar leat times
model_df['LeadTimeSemanas']=model_df['Politica_PQN'].map({'MTS':6, 'MTO':10, np.nan:10})

In [55]:
# configurar periodo de revisión
model_df['Periodo_Revision'] = 2

<p>Calcularemos el Safety stock para un modelo periódico (OUL) con un nivel de sercicio (CSL) del 95%, bajo la siguiente fórmula</p>
</br>

$$ SafetyStock = K_{95\%}RMSE_{semanal}\sqrt{lt+R} $$

In [56]:
# Asumiremos un modelo
model_df['SS'] = 1.64*model_df['RMSE_SEMANAL']*(model_df['LeadTimeSemanas'] + model_df['Periodo_Revision'])**0.5

In [57]:
# Remover el SS de productos que no sean MTS
model_df['SS'] = model_df.apply(lambda x: x['SS'] if x['SuggestedInvPolicy']=='MTS' else 0,axis=1)

In [58]:
print(model_df.shape)
model_df.head()

(1746, 10)


,COD_EMPRESA,Inventory,SuggestedInvPolicy,Politica_PQN,PRODUCTO,TotalOnStock,RMSE_SEMANAL,LeadTimeSemanas,Periodo_Revision,SS
3,GMB,SI,MTS,MTO,940-9001,609.3,58.986716,10,2,335.111002
4,UK,SI,DSC,MTO,NaN,NaN,NaN,10,2,0.000000
5,SLU,SI,MTS,MTO,940-9001,327.8,25.893616,10,2,147.104911
6,GMB,SI,MTS,MTO,940-8001,27.7,42.886260,10,2,243.642274
7,UK,SI,DSC,MTO,NaN,NaN,NaN,10,2,0.000000


<h3>Programación matemática</h3>
<p>Este modelo genera las cantidades sugeridas a pedir a la planta en función del forecast de venta.</p>
<p><strong>Conjuntos</strong>:</p>
<ul>
    <li>b: conjunto de bodegas</li>
    <li>t: conjunto de periodos</li>
</ul>
<p><strong>Parámetros</strong>:</p>
<ul>
    <li>$ HE $: costo unitario de mantener el producto en bodega. Surge del costo del producto y el costo de oportunidad por periodo</li>
    <li>$ CP $: costo de colocar una orden de abastecimiento para un sku totalizando las tres bodegas</li>
    <li>$ I_{bo} $: Inventario inicial en la bodega b</li>
    <li>$ SS_{b} $: Inventario de seguridad para el final del periodo. El modelo intentará mantener el cierre de inventario sobre esta cifra en on_hand</li>
    <li>$ LM $: Lote mínimo de producción, asegurará que la suma del pedido de las bodegas no esté por debajo de este valor.</li>
    <li>$ MPO $: Pedido mínimo: Asegura que no se sugieran pedidos de menos de este valor.</li>
    <li>$ F_{bt} $: Pronóstico de ventas del producto en la bodega b durante el periodo t</li>
    <li>$ A_{bt} $: Llegadas calculadas de producto a la bodega b durante el periodo t</li>
    <li>$ LT_{b} $: Lead time calculado entre las plantas y las bodegas</li>
</ul>
<p><strong>Variables</strong>:</p>
<ul>
    <li>$ X_{bt} $: Cantidad de producto a pedir para la bodega b durante el periodo t</li>
    <li>$ M_{bt} $: Binaria, 1 si se coloca el pedido mínimo (30 metros?) durante el periodo t para la bodega b; 0 en otr caso</li>
    <li>$ P_{t} $: Binary, 1 si se coloca un pedido desde cualquiera de las bodegas durante el periodo t. Esta se usará para garantizar el pedido mínimo para producción</li>
    <li>$ I_{bt} $: Variable de Estado, Cantidad de producto al cierre del periodo t en la bodega b</li>
</ul>
<p><strong>Función objetivo</strong>:</p>
<p>La siguiente expresión minimiza el costo total relevante compuesto por el costo total de colocar las órdenes de pedido y el costo de mantener el inventario:</p>
<p>$ min \sum_{bt}{CP*P_{t} + HE*X_{bt}} $ </p>
<p><strong>Sujeto a</strong>:</p>
<p><strong>Resticción de Inventario de seguridad:</strong> Evita que quede inventario por debajo del SS luego del leadtime</p>
<p>$ I_{bt} \geq {SS_{bt}} \forall{t > LT_{b}}$</p>
<p><strong>Flujos de inventario:</strong> Mantiene el balance de masa del inventario. Para el primer caso, se encarga de inicializar el inventario, en el segundo del flujo durante el lead time y, finalmente, el flujo cuando los pedidos colocados empiezan a llegar</p>
<p>$ I_{bt} = I_{bo} - F_{bt} + A_{bt} \forall{b}; t=0 $</p>
<p>$ I_{bt} = I_{bt} - F_{bt} + A_{bt} \forall b ; \forall 1 < t < LT $</p>
<p>$ I_{bt} = I_{bt} - F_{bt} + A_{bt} + X_{bt} \forall b ; \forall t \geq LT $</p>

In [59]:
def calcular_pedidos(holding_cost:float, order_cost:float, initial_inventory:list, safety_stock:list, min_batch_size:float, min_po:float, forecast:list, arrivals:list, leadtime:list, max_time_seconds=0):
    
    # variable Xbt: Cantidad pedida para la bodega b durante el periodo t
    Xbt = [[pu.LpVariable(name=f'Xb{b}t{t}',lowBound=0, upBound=3000) for t in range(len(forecast[0]))] for b in range(len(forecast))]
    
    # variable Pbt: 1 si se coloca pedido durante el periodo t; 0 en otro caso
    Pt = [pu.LpVariable(f'Pt{t}', cat='Binary') for t in range(len(forecast[0]))]
    
    Mbt = [[pu.LpVariable(name=f'Mb{b}t{t}', cat='Binary') for t in range(len(forecast[0]))] for b in range(len(forecast))]
    
    # variable Ibt: cantidad de inventario en la bodega b al final del periodo t
    Ibt = [[pu.LpVariable(name=f'Ib{b}t{t}') for t in range(len(forecast[b]))] for b in range(len(forecast))]
    
    # declarar el problema
    prob = pu.LpProblem("Suggested_PO", pu.LpMinimize)
    # Colocar funcion objetivo
    prob += (pu.lpSum([order_cost*Pt[t] for t in range(len(forecast[0]))] + 
                      [holding_cost*Ibt[b][t] for t in range(len(forecast[0])) for b in range(len(forecast))]),
                       'Costo total relevante')
    
    #prob += (pu.lpSum([Ibt[b][t] for t in range(len(forecast[0])) for b in range(len(forecast))]), 'Metros almacenados')

    # restriccion para evitar quedar bajo el SS
    # para cada bodega
    for b in range(len(forecast)):
        # para cada periodo
        for t in range(leadtime[b], len(forecast[b])):
            # obligat a que Ibt termine sobre el SS
            prob += (Ibt[b][t] >= safety_stock[b], 
                     f'terminar inventario al final de {t} sobre {safety_stock[b]} en la bodega {b}')

    # restriccion de flujo de inventarios
    # para cada bodega
    for b in range(len(forecast)):
        
        # el inventario final es el inicial menos el forecast + las llegadas programadas
        prob += (Ibt[b][0] == initial_inventory[b] - forecast[b][0] + arrivals[b][0], 
                 f'inventario en la bodega {b} al final del periodo {0}')        
        
        # para cada periodo
        for t in range(1, len(forecast[0])):
            if t < leadtime[b]:
                # el inventario final es el cierre del periodo anterior menos el forecast + las llegadas programadas
                prob += (Ibt[b][t] == Ibt[b][t-1] - forecast[b][t] + arrivals[b][t],
                         f'inventario en la bodega {b} al final del periodo {t}')
            else:
                # el inventario final es el cierre del periodo anterior menos el forecast + las llegadas programadas + pedidos colocados un lead time antes
                prob += (Ibt[b][t] == Ibt[b][t-1] - forecast[b][t] + arrivals[b][t] + Xbt[b][t-leadtime[b]], 
                         f'inventario en la bodega {b} al final del periodo {t}') 

    # restricciones de pedidos mínimos
    
    # para cada bodega
    for b in range(len(forecast)):
        # para cada periodo
        for t in range(len(forecast[0])):        
            # pedir sobre el pedido mínimo
            prob += (Xbt[b][t] >= min_po*Mbt[b][t], f'pedir sobre el pedido minimo durante {t} en la bodega {b}')
            # habilitar capacidad
            prob += (Xbt[b][t] <= 10000*Mbt[b][t], f'pedir menos de la capacidad durante {t} en la bodega {b}')
    
    # La suma de los pedidos debe superar el lote minimo de producción
    for t in range(len(forecast[b])):        
        prob += (pu.lpSum([Xbt[b][t] for b in range(len(forecast))]) >= min_batch_size*Pt[t], 
                 f'superar el lote minimo de produccion durante {t}')  
        prob += (pu.lpSum([Xbt[b][t] for b in range(len(forecast))]) <= 10000*Pt[t], 
                 f'No emitir pedido si no se activa el pedidodurante {t}') 
    
    #print(prob)
    
    # prob.solve(pu.PULP_CBC_CMD(timeLimit=max_time_seconds))
    if max_time_seconds != 0:
        prob.solve(pu.GLPK_CMD(options=['--mipgap', '0.35', f'--tmlim {max_time_seconds}']))
    else:
        prob.solve(pu.GLPK_CMD(options=['--mipgap', '0.35']))
    
    pedidos = [[Xbt[b][t].value() for t in range(len(forecast[b]))] for b in range(len(forecast))]    
    inventario = [[Ibt[b][t].value() for t in range(len(forecast[b]))] for b in range(len(forecast))]
    
    return pedidos, inventario, forecast, arrivals


In [60]:
# Ejemplo
ejemplo = calcular_pedidos(holding_cost=7*0.06, order_cost = 150,
                           initial_inventory=[25,12,13],
                           safety_stock=[15,13,15],
                           min_batch_size=500,
                           min_po = 30,
                           forecast=[[30,20,30,50,50], [25,23,12,12,10], [10,10,10,0,10]],
                           arrivals=[[0,0,0,0,0], [0,0,0,0,0], [0,0,0,0,0]],
                           leadtime=[2, 2, 2])

In [61]:
ejemplo

([[375.0, 0.0, 0.0, 0.0, 0.0],
  [83.0, 0.0, 0.0, 0.0, 0.0],
  [42.0, 0.0, 0.0, 0.0, 0.0]],
 [[-5.0, -25.0, 320.0, 270.0, 220.0],
  [-13.0, -36.0, 35.0, 23.0, 13.0],
  [3.0, -7.0, 25.0, 25.0, 15.0]],
 [[30, 20, 30, 50, 50], [25, 23, 12, 12, 10], [10, 10, 10, 0, 10]],
 [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]])

In [68]:
def procesar_stock(stock_code:str, holding_cost:float, order_cost:float, max_time_seconds=0)->pd.DataFrame:
    # Forecast
    temp = model_df[model_df['PRODUCTO']==stock_code]

    # Ajustar forecast en caso que alguna bodega no tenga forecast
    max_len = len(forecast.columns)    
    
  
    # Cuadrar los forecast
    try:
        forecast_gmb = list(forecast.loc[('GMB', stock_code)])
    except:
        forecast_gmb = np.zeros(max_len).tolist()
    
    try:
        forecast_slu = list(forecast.loc[('SLU', stock_code)])
    except:
        forecast_slu = np.zeros(max_len).tolist()
        
    try:   
        forecast_ukn = list(forecast.loc[('UK', stock_code)])
    except:
        forecast_ukn = np.zeros(max_len).tolist()

    forecast_list = [forecast_gmb,forecast_slu,forecast_ukn]
    
    # Safety Stock
    ss = list()
    for b in ['GMB', 'SLU', 'UK']:
        try:
            ss.append(list(temp.groupby(['COD_EMPRESA'])[['SS']].min().loc[b])[0])
        except:
            ss.append(0.0)
    #print('SS', ss)
    # initial inventory
    ii = list()
    for b in ['GMB', 'SLU', 'UK']:
        try:
            ii.append(list(temp.groupby(['COD_EMPRESA'])[['TotalOnStock']].min().loc[b])[0])
        except:
            ii.append(0.0)
    #print('Ioh0', ii)
    # arrivals
    ar = list()
    for b in ['GMB', 'SLU', 'UK']:
        try:
            #ar.append([0.0]*max_len) 
            ar.append(arrivals.loc[(b, stock_code)].to_list())
        except:
            ar.append([0.0]*max_len)    
    #print('arrivals', ar)
    

    if pqn_policy_dict[stock_code] =='MTS':
        min_batch_size_temp = 30
    else:
        min_batch_size_temp = 500

            

    pedidos = calcular_pedidos(holding_cost=holding_cost, 
                               order_cost = order_cost,
                               initial_inventory=ii,
                               safety_stock=ss,
                               min_batch_size=min_batch_size_temp,
                               min_po = 30,
                               forecast=forecast_list,
                               arrivals=ar,
                               leadtime=[12, 12, 12], 
                               max_time_seconds=max_time_seconds)
    
    return pd.DataFrame({'STOCK':[stock_code]*len(pedidos[0][0]),
                         'GMB_PO': pedidos[0][0], 
                         'SLU_PO':pedidos[0][1], 
                         'UK_PO':pedidos[0][2], 
                         'GMB_Ioh': pedidos[1][0],                          
                         'SLU_Ioh':pedidos[1][1], 
                         'UK_Ioh':pedidos[1][2], 
                         'GMB_fct': pedidos[2][0], 
                         'SLU_fct':pedidos[2][1], 
                         'UK_fct':pedidos[2][2], 
                         'GMB_arl': pedidos[3][0], 
                         'SLU_arl':pedidos[3][1], 
                         'UK_arl':pedidos[3][2]})

In [69]:
# Obtener la lista relevante de productos
stock_list = list(model_df['PRODUCTO'].unique())
print(len(stock_list))

583


In [70]:
# Obtener la lista de políticas
temp = model_df.groupby(['PRODUCTO'])[['Politica_PQN']].first()
pqn_policy_dict = {x : temp.loc[x]['Politica_PQN'] if temp.loc[x]['Politica_PQN']!=None else 'MTO' for x in temp.index}
print(len(pqn_policy_dict))

582


In [71]:
# Calcular los pedidos sugeridos usando el modelo
results = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_po = {executor.submit(procesar_stock, 
                                       stock_code=stock, 
                                       holding_cost=0.06, 
                                       order_cost=25, 
                                       max_time_seconds=120): stock for stock in stock_list}
    
    for future in concurrent.futures.as_completed(future_to_po):
        stock = future_to_po[future]
        results.append(future.result())
print('Se generaron', len(results),'planes de compra')

KeyError: nan

In [ ]:
# Agregar los periodos a cada tarea
for result in results:
    result['Semana']= model_periods
    
# Unir los resultados
results_df = pd.concat(results).copy()

In [ ]:
# colocar las fechas
results_df = pd.merge(left=results_df,
                      right=periodos_df.groupby(['Semana'])[['Fecha']].min().reset_index(),
                      left_on=['Semana'],
                      right_on=['Semana'],
                      how='left')

# Colocar datos en formato vertical para PowerBI
results_df = results_df.melt(id_vars=['STOCK', 'Semana', 'Fecha'],
                             value_vars=['GMB_PO', 'SLU_PO', 'UK_PO', 'GMB_Ioh', 'SLU_Ioh', 'UK_Ioh',
                                         'GMB_fct', 'SLU_fct', 'UK_fct', 'GMB_arl', 'SLU_arl', 'UK_arl'],
                             value_name='Meters')

# Separar variables
results_df['COD_EMPRESA']=results_df['variable'].apply(lambda x: str(x).split('_')[0])
results_df['variable']=results_df['variable'].apply(lambda x: str(x).split('_')[1])

results_df = results_df.pivot_table(values=['Meters'],
                                    columns=['variable'],
                                    index=['COD_EMPRESA', 'STOCK', 'Semana', 'Fecha'],
                                    aggfunc=sum).reset_index()
# Organizar títulos
results_df.columns = [x[0] if x[1]=='' else x[1] for x in results_df.columns]

print(results_df.shape)
results_df.head()

In [ ]:
results_df['on_floor'] = results_df['Ioh'].apply(lambda x: x if x>=0.0 else 0.0)
results_df['Backorder'] = results_df['Ioh'].apply(lambda x: x if x<0.0 else 0.0)
results_df.rename(columns={'COD_EMPRESA':'DIVISION'}, inplace=True) 
results_df['TipoDato'] = 'Proyectado'

In [ ]:
# Guardar para el reporte
results_df.to_parquet(f'Y:\\PlaneacionAbastecimiento\\data\\proyected_inventories\\ProjectedInventory_EUR_{hoy}.parquet', index=False)

In [ ]:
results_df = pd.read_parquet(f'Y:\\PlaneacionAbastecimiento\\data\\proyected_inventories\\ProjectedInventory_EUR_{20220407}.parquet', index=False)



In [ ]:
# Ir a Power BI

In [ ]:
# Prueba
stock = '940-9001'
temp = procesar_stock(stock_code=stock, holding_cost=0.01, order_cost=12)

In [ ]:
warehouses = ['GMB', 'SLU', 'UK']
variables =['Ioh', 'PO', 'fct']
plt.figure(figsize=(12,12))
pos = 1
for var in variables:
    for wh in warehouses:            
        plt.subplot(3,3, pos, label=f'{wh}_{var}', title= f'{wh}_{var}')
        plt.plot(temp[f'{wh}_{var}'])
        pos+=1
        #plt.legend()    
plt.show()

In [1]:
for i in range(10):
    print("Hola Alexander")

Hola Alexander
Hola Alexander
Hola Alexander
Hola Alexander
Hola Alexander
Hola Alexander
Hola Alexander
Hola Alexander
Hola Alexander
Hola Alexander


In [8]:
variable = "1"
if variable=="1":
    print("La variable es 1")
    print("ESta es otra linea")
else:
    print("La variable no es 1")

La variable es 1
ESta es otra linea
